# Data Science Capstone Project.

This notebook will contain the Data Science capstone project involving location data of neighbourhoods in Toronto, Canada

In [46]:
import pandas as pd
import numpy as np

In [47]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Part 1 Web Scraping and Data Cleaning

Get table from wikipedia wep page and read into a pandas dataframe using read_html method. 

In [48]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = pd.read_html(url)
pc_can = wiki_page[0]
pc_can.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


We can see that the Neighborhood column has some null values. We can change these to Not assigned in order to make some further checks

In [49]:
pc_can['Neighborhood'].fillna('Not assigned', inplace = True)
pc_can.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [50]:
pc_can.shape

(180, 3)

We see that the original data has 180 rows. We know that we want dont need rows where Borough is not assigned. However we also need to check for rows where neighborhood is not assigned but there may be a borough assigned,

In [51]:
count = 0
for index, row in pc_can.iterrows():
    if (row['Neighborhood'] != 'Not assigned') & (row['Borough'] == 'Not assigned'):
        count += 1
    
if count == 0:
    print('There are no instances of neighbourhoods Not assigned and Boroughs assigned')
else:
    print('Number of instances where neighbourhoods not assigned but Borough is assigned  =:', count)

There are no instances of neighbourhoods Not assigned and Boroughs assigned


Now we know that we know that the rows where neighborhood is not assigned matches with the rows where Borough is not assigned. Therefore we can simply make a dataframe without rows where borough is not assigned. 
First we can confirm how many rows like this exist. 

In [52]:
pc_can.groupby('Borough').count()

,Postal Code,Neighborhood
Borough,,
Central Toronto,9,9
Downtown Toronto,19,19
East Toronto,5,5
East York,5,5
Etobicoke,12,12
Mississauga,1,1
North York,24,24
Not assigned,77,77
Scarborough,17,17


In [53]:
pc_can = pc_can.loc[pc_can['Borough'] != 'Not assigned']
pc_can.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Now that we have removed unwamted rows and verified that the Neighborhood column is filled in we need to ensure that there is no repeated postal code. We can compare the number of unique postal codes with the number or rows. If they are the same it means each row corresponds to a unique postal code.

In [54]:
if (pc_can['Postal Code'].nunique()) == pc_can.shape[0]:
    print('There are no repeated postal codes')
else:
    print('There are repeated postal codes')

There are no repeated postal codes


Finally we can reset the index and print the cleaned dataframe head and shape

In [55]:
pc_can.reset_index(drop = 'True', inplace = True)
pc_can.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [56]:
pc_can.shape

(103, 3)

## Part 2 Get GPS Coordinates

Here we will read a csv file containing the GPS co-ordinates into a dataframe and join using the postal code column to create one dataframe with latitude and longitude.

In [57]:
gps = pd.read_csv('https://cocl.us/Geospatial_data')
gps.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [58]:
pc_can = pd.merge(pc_can, gps, on ='Postal Code', how='left')
pc_can.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [59]:
pc_can.tail()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999


In [60]:
pc_can.shape

(103, 5)

## Part 3 Cluster Neighbourhoods
This section will take postal codes associated with toronto and retrieve nearby venues from foursquare. The mean number of occurences of each venue categories in each neighbourhood will be passed to a clustering algorithm to cluster the neighborhoods. The 10 most popular venue catergories for each neighborhood will then be used along with the cluster labels to propose some reasoning for the clusters. It was found that Toronto mostly has a wide selection of cafes, resturants and amenities associated with an urban city. Much smaller clusters were formed for neighborhoods with more green spaces or open areas like parks and pools. 

In [61]:
!pip install folium

In [62]:
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

We can create a dataframe that made up of Boroughs that contain "Toronto"

In [63]:
pc_toronto = pc_can.loc[pc_can['Borough'].str.contains('Toronto')]
pc_toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


Create credentials for accessing foursquare data.

In [64]:
CLIENT_ID = 'ENKDW1AA3EF4LLRXJYEIR1XURLBIOMNNTZOGIOAKK1JYH1VK' # your Foursquare ID
CLIENT_SECRET = 'P0VDY3XYEEZ4FR0CVCGIK3BHHNNZCJTTMAWE5IKYH1T0XPOT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ENKDW1AA3EF4LLRXJYEIR1XURLBIOMNNTZOGIOAKK1JYH1VK
CLIENT_SECRET:P0VDY3XYEEZ4FR0CVCGIK3BHHNNZCJTTMAWE5IKYH1T0XPOT


Create a function to retreive nearby venues for each postal codes using the latitude and Longitude in the pc_toronto dataframe

In [65]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [67]:
toronto_venues = getNearbyVenues(names = pc_toronto['Neighborhood'],
                                latitudes = pc_toronto['Latitude'],
                                longitudes = pc_toronto['Longitude'] )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [68]:
print(toronto_venues.shape)
toronto_venues.head()

(1190, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [69]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues.groupby('Neighborhood').count()

There are 224 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,50,50,50,50,50,50
Christie,17,17,17,17,17,17
Church and Wellesley,50,50,50,50,50,50
"Commerce Court, Victoria Hotel",50,50,50,50,50,50
Davisville,36,36,36,36,36,36


For each unique venue type use one hot encoding. This will allow venue type to be used later in the clustering model.

In [70]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
first_col = toronto_onehot.pop('Neighborhood')
toronto_onehot.insert(0,'Neighborhood', first_col)
print(toronto_onehot.shape)
toronto_onehot.head()


(1190, 224)


,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group by neighbourhood to consolidate information for each unique neighborhood and find the mean of each category for each neighborhood.

In [71]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.066667,0.066667,0.066667,0.133333,0.2,0.066667,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.02


Now we can determine the 10 most common venue categories for each neighbourhood. Later this can be used to give some insights to the resulting clusters and possibly help the user assign meaning or labels to the clusters found.

In [72]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [73]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Beer Bar,Cheese Shop,Café,Seafood Restaurant,Japanese Restaurant,Beach
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Nightclub,Burrito Place,Italian Restaurant,Intersection,Stadium,Restaurant,Bar
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Skate Park,Butcher,Fast Food Restaurant,Farmers Market,Auto Workshop,Burrito Place,Spa,Brewery,Pizza Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport,Harbor / Marina,Plane,Coffee Shop,Sculpture Garden,Bar,Boat or Ferry,Airport Terminal
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Burger Joint,Café,Ice Cream Shop,Department Store,Salad Place,Ramen Restaurant


Cluster neighbourhoods using the toronto_grouped dataframe that contains each neighbourhood and the mean occurence of each venue category.

In [74]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Create a new dataframe with postal code, borough, neighbourhood, lat, long, cluster label and 10 most common venues and then visualise on a map of toronto. 

In [75]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = pc_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Restaurant,Theater,Yoga Studio,Mexican Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Cafeteria,Park
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Café,Ramen Restaurant,Bookstore,Clothing Store,Cosmetics Shop,Coffee Shop,Tea Room,Theater,Restaurant,Middle Eastern Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Gastropub,Coffee Shop,Cocktail Bar,American Restaurant,Cosmetics Shop,Creperie,Park,Diner,Restaurant
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Pub,Trail,Health Food Store,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [76]:
latitude = 43.6532
longitude = -79.3832
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Regent Park, Harbourfront",0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Restaurant,Theater,Yoga Studio,Mexican Restaurant
4,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Cafeteria,Park
9,"Garden District, Ryerson",0,Café,Ramen Restaurant,Bookstore,Clothing Store,Cosmetics Shop,Coffee Shop,Tea Room,Theater,Restaurant,Middle Eastern Restaurant
15,St. James Town,0,Café,Gastropub,Coffee Shop,Cocktail Bar,American Restaurant,Cosmetics Shop,Creperie,Park,Diner,Restaurant
20,Berczy Park,0,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Beer Bar,Cheese Shop,Café,Seafood Restaurant,Japanese Restaurant,Beach
24,Central Bay Street,0,Coffee Shop,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Burger Joint,Café,Ice Cream Shop,Department Store,Salad Place,Ramen Restaurant
25,Christie,0,Grocery Store,Café,Park,Restaurant,Baby Store,Nightclub,Coffee Shop,Candy Store,Athletics & Sports,Italian Restaurant
30,"Richmond, Adelaide, King",0,Coffee Shop,Café,Concert Hall,Steakhouse,American Restaurant,Pizza Place,Restaurant,Gluten-free Restaurant,Burrito Place,Smoke Shop
31,"Dufferin, Dovercourt Village",0,Pharmacy,Bakery,Music Venue,Brewery,Café,Bar,Bank,Supermarket,Pool,Middle Eastern Restaurant
36,"Harbourfront East, Union Station, Toronto Islands",0,Coffee Shop,Aquarium,Brewery,Plaza,Park,Café,Hotel,Sports Bar,Baseball Stadium,Basketball Stadium


In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Roselawn,1,Pool,Garden,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [79]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,The Beaches,2,Pub,Trail,Health Food Store,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
83,"Moore Park, Summerhill East",2,Park,Playground,Summer Camp,Restaurant,Yoga Studio,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
91,Rosedale,2,Park,Playground,Trail,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [80]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,"Forest Hill North & West, Forest Hill Road Park",3,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Yoga Studio,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [81]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Lawrence Park,4,Park,Swim School,Bus Line,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


From this analysis we see that cluster 0 is the most popular cluster and represents areas that seem to be more urban and are characterised by high amounts of cafes and diverse resturants bars and amenities like gyms and yoga. This is expected in a densely populated city lie toronto.
The remaining clusters are smaller and are made up of areas with more open spaces like pools or parks.